#Ingestion del archivo movie_cast.json


### Paso 1 - Leer el archivo JSON usando "DataframeReader"

In [0]:
dbutils.widgets.text("p_environment", "")
v_enviroment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run 
"../includes/common_functions"

In [0]:
%run
"../includes/configuration"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType

In [0]:
movie_casts_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("personId", IntegerType(), True),
    StructField("characterName", StringType(), True),
    StructField("genderId", IntegerType(), True),
    StructField("castOrder", IntegerType(), True)
])

In [0]:
movie_casts_df = spark.read \
    .schema(movie_casts_schema) \
    .option("multiLine", True) \
    .json(f"{bronze_folder_path}/{v_file_date}/movie_cast.json")


## Parte 2 : Renombrar las columnas y añadir nuevas columnas
##### 1. "personId" renombrar a "person_id"
##### 2. Agregar las columnas "Ingestion_date" y "environment"
##### 3. Agregar la columna "name" a partir de la concatenación de "forename" y "surname"

In [0]:
from pyspark.sql.functions import col,concat,current_timestamp,lit



In [0]:
movies_casts_with_columns_df = movie_casts_df \
                                .withColumnsRenamed({"movieId": "movie_id",
                                                     "personId": "person_id",
                                                     "characterName": "character_name"}) \
                                .withColumn("ingestion_date", current_timestamp()) \
                                .withColumn("enviroment", lit("Production")) \
                                .withColumn("file_date", lit(v_file_date))






### Paso 3. Eliminar las columnas no deseadas de Dataframe

In [0]:
from pyspark.sql.functions import col

In [0]:
movies_casts_final_df = movies_casts_with_columns_df.drop(col("castOrder"),col("genderId"))

In [0]:
#overwrite_partition (movies_casts_final_df, "movie_silver", "movie_cast", "file_date")

In [0]:
#movies_casts_final_df.write.mode("append").format("parquet").saveAsTable("movie_silver.movies_cast")

In [0]:
merge_key = 'tgt.movie_id = src.movie_id and tgt.person_id = src.person_id'
merge_delta_data(movies_casts_final_df,"/mnt/moviehistory7809/silver","movie_silver", "movie_cast", "file_date", merge_key)

In [0]:
%sql

select file_date,count(1)
from movie_silver.movie_cast
group by file_date


In [0]:
dbutils.notebook.exit("Success No 7")